In [1]:
import pandas as pd
import statsmodels.api as sm
import datetime
import sklearn as sk
%ls

Add_ints.ipynb     export.csv          mancheck.csv     target_df.csv
Adjjkd_ints.ipynb  exsup.csv           merged.csv       Untitled.ipynb
better_exsup.csv   exsup_merged.csv    merged_midi.csv  us_presidents.csv
BuildBase.ipynb    hm.csv              NMC.csv          WimmerMin.csv
check.csv          late_20_merged.csv  notebook.tex     WimmerPrepared.csv
Draft.ipynb        maildit.csv         saved.csv        zwei.csv


# Load Wimmer dataset to use as base df

In [2]:
wim = pd.read_csv('WimmerMin.csv')
# Remove all non-coldwar entries
mask = wim.year > 1945 
wim = wim[mask]
mask = wim.year < 1992
wim = wim[mask]
# Wimmers war data, when ignored, will cause duplicate rows to appear, remove them
wim.drop_duplicates(subset=['year', 'cowcode'], keep='first', inplace=True)


### Add US political information (Dummies for presidents and parties)

In [3]:
# Convert start to datetime
pres = pd.read_csv('us_presidents.csv')
pres['start'] = pd.to_datetime(pres['start'])
# Filter out non relevant presidents
mask = pres['start'] > datetime.date(1945, 1, 1)
pres = pres[mask]
mask = pres['start'] < datetime.date(1992, 1, 1)
pres = pres[mask]
pres['start'] = pres.start.dt.year
# Set turman start to 1946 for data reasons
pres = pres[['start', 'president', 'party']]
pres[:1].start = pres[:1].start +1
dummy_pres = pres['president'].str.get_dummies()
dummy_party = pres['party'].str.get_dummies()
pres = pd.concat([pres, dummy_pres], axis=1)
pres = pd.concat([pres, dummy_party], axis=1)
pres = pres[['start', 'Dwight D. Eisenhower', 'George H. W. Bush', 
'Gerald Ford', 'Harry S. Truman', 'Jimmy Carter',
'John F. Kennedy', 'Lyndon B. Johnson', 'Richard Nixon',
'Ronald Reagan', 'Democratic', 'Republican']]
    
wim = wim.merge(pres, left_on='year', right_on='start', how='left')
wim[[
    'Dwight D. Eisenhower', 'George H. W. Bush', 
    'Gerald Ford', 'Harry S. Truman', 'Jimmy Carter',
    'John F. Kennedy', 'Lyndon B. Johnson', 'Richard Nixon',
    'Ronald Reagan', 'Democratic', 'Republican']] = wim[
    [
        'Dwight D. Eisenhower', 'George H. W. Bush', 
        'Gerald Ford', 'Harry S. Truman', 'Jimmy Carter',
        'John F. Kennedy', 'Lyndon B. Johnson', 'Richard Nixon',
        'Ronald Reagan', 'Democratic', 'Republican'
    ]].ffill()



/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### Add US / SU GDP and CINC scores

In [4]:
# Add GDP related variables
# Split of both countries
us_only = wim['cowcode'] == 2
su_only = wim['cowcode'] == 365

rus_gdp = wim[su_only]
rus_gdp = rus_gdp[['gdppc', 'gdppcl', 'year']]
rus_gdp.columns = ['SU_gdppc', 'SU_gdppcl', 'year']

us_gdp = wim[su_only]
us_gdp = us_gdp[['gdppc', 'gdppcl', 'year']]

us_gdp
us_gdp.columns = ['US_gdppc', 'US_gdppcl', 'year']

gdp_merged = pd.concat([rus_gdp, us_gdp], axis=1)
gdp_merged 


# Get CINC scores
cinc = pd.read_csv('NMC.csv')
mask = cinc['year'] < 1992
cinc = cinc[mask]
mask = cinc['year'] > 1945
cinc = cinc[mask]
cinc = cinc[['ccode', 'year', 'milex', 'cinc']]

# Get MIl_Ex as sperate variable, 
# (as the Soviet Union had a large army despite a lagging economy)
# Again, split of both countries
us_only = cinc['ccode'] == 2
su_only = cinc['ccode'] == 365

rus_cinc = cinc[su_only]
rus_cinc = rus_cinc[['milex', 'cinc', 'year']]
rus_cinc.columns = ['SU_milex', 'SU_cinc', 'year']

us_cinc = cinc[us_only]
us_cinc = us_cinc[['milex', 'cinc', 'year']]
us_cinc.columns = ['US_milex', 'US_cinc', 'year']

rus_cinc.reset_index(inplace=True)
us_cinc.reset_index(inplace=True)  

cinc_merged = pd.concat([rus_cinc, us_cinc], axis=1)
cinc_merged = cinc_merged[['SU_milex', 'SU_cinc', 'US_milex', 'US_cinc', 'year']]
cinc_merged.reset_index(inplace=True)
gdp_merged.reset_index(inplace=True)


gpd_cinc_merged = pd.concat([gdp_merged, cinc_merged], axis=1)

# Remove duplicate columns. 
c = pd.Index(['SU_gdppc', 'SU_gdppcl', 'year', 'US_gdppc', 'US_gdppcl',
       'year', 'SU_milex', 'SU_cinc', 'US_milex', 'US_cinc', 'year',
       'year'])

gpd_cinc_merged = gpd_cinc_merged.loc[:, ~gpd_cinc_merged.columns.duplicated()]

#Merge new dataframe into Wimmers dataset
wim = wim.merge(gpd_cinc_merged, left_on='year', right_on='year', how='left')

# Add cinc and milex variables for indivual nations as well. 
wim = wim.merge(cinc, left_on=['year', 'cowcode'], right_on=['year', 'ccode'], how='left')

#Save the file
wim.to_csv('WimmerPrepared.csv')


# Prepare external support DF

## WARNING - This block contains a very slow groupby operation, which will take +/- 15 minutes

In [5]:
exsup = pd.read_csv('exsup.csv')
# Remove all non-coldwar entries
mask = exsup['ywp_year'] < 1992
exsup = exsup[mask]
exsup['United States', 'Soviet Union'] = 0
#remove non US / SU interventions
dummies = exsup['external_name'].str.get_dummies(sep=';')
dummies = dummies[['United States', 'Soviet Union']]
dummies.columns = ['United States', 'Soviet Union']
exsup = pd.concat([exsup, dummies], axis=1)
# Drop duplicates
exsup.drop_duplicates(inplace=True)

# Add support specific dummies
old = ['external_type__X', 'external_type__L', 'external_type__Y',
         'external_type__W', 'external_type__M', 'external_type__T',
         'external_type__$', 'external_type__I', 'external_type__O', 
         'external_type__U']
new = [
    'troops', 'joint_ops', 'terr', 'weapons', 'logistics',
    'training', 'eco', 'intel', 'other', 'unknown'
]

us_support_cols = ['US_' + x for x in old]
su_support_cols = ['SU_' + x for x in old]

for col in us_support_cols:
    exsup[col] = 0
for col in su_support_cols:
    exsup[col] = 0
   

# # Build dict to rename columns
# coldict = {}
# index = 0
# for col in old:    
#     coldict[col] = new[index]
#     index +=1
# # Rename columns
# exsup.rename(coldict, inplace=True)


for index, row in exsup.iterrows():
    if row['Soviet Union'] == 1:
        nation = 'SU_'
        for col in old:
            exsup[index: index + 1][nation + col] = int(row[col])
    if row['United States'] == 1:
        nation = 'US_'
        for col in old:
            exsup[index: index + 1][nation + col] = int(row[col])
            

# Save to csv
exsup.to_csv('export.csv')
exsup_backup = exsup
# Merge dataframe per year-country combination
exsup = exsup.groupby(['locationid1', 'ywp_year']).max()
exsup.to_csv('exsup_merged.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


TypeError: '<' not supported between instances of 'tuple' and 'str'

## Merge UCDP external support into the BaseDataset

In [ ]:
# Reset index so 'locationid1' becomes available again. 
exsup.reset_index(inplace=True)

In [ ]:
merged = wim.merge(exsup, left_on=['cowcode', 'year'], right_on=['locationid1', 'ywp_year'], how='left')
# External exists is a value that we dont want Null values for, ffill them. 
merged['external_exists'].fillna(value=0, inplace=True)

# # Add dummies for US and RU interventions
# intervention_dummies = merged['external_name'].str.get_dummies()
# merged = pd.concat([merged, intervention_dummies], axis=1)



In [ ]:
late20 = wim.year > 1975
late20 = wim[late20]

In [ ]:
for x in late20.columns:
    print(x)

In [ ]:
logit = sm.Logit(late20['United States'], late20[['Soviet Union', 'cinc']], missing='drop')
#logit = sm.Logit(late20['Soviet Union'], late20[['onatind', 'ssafrica']], missing='drop')

#'anarc', 'ointrap', 'instab'

In [ ]:
results = logit.fit(missing='drop')

In [ ]:
print(results.summary())

In [ ]:
# MATCHES - US
[['Soviet Union', 'poldisc']]

In [ ]:
# Matches - SU

In [ ]:
for col in wim.columns:
    print(col)